In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import re

from sklearn.linear_model import LinearRegression
import seaborn

import pyperclip

# import tkinter as tk
# import tkinter.messagebox as messagebox

In [5]:
pd.set_option('display.max_columns', 50)

### ここだけ手動で設定。変更がないか毎度確認すること

In [11]:
subject_name_list = ["kumakura","kim","souma","fujii","tubota","toki"]
phase_name_list = ["rest", "practice", "boredom", "flow", "ultra", "overload"]
target_columns = ["bpm", "ibi", "lf", "hf","lf/hf","sdsd","rmssd","pnn20","pnn50","concentration_FC3","concentration_FC4","concentration_FCz","concentration_mean","FC3","FC4","FCz","mean_3ch","lf_diff", "hf_diff","lfhf_diff","hr_mad","sd1","sd2","s","sd1/sd2","breathingrate","lf_nu","hf_nu"]
target_phases = ["boredom","flow","ultra","overload"]
survey_items = ["difficulty","fast","fulfillment","absorb","questionnaire_average"]

folder_name = "999_2_for_paper"
pyperclip.copy(f"{folder_name}")

# tk.Tk().withdraw()
# messagebox.showinfo(f'{folder_name}フォルダを作成してください')



### 分析対象のファイルのパスを正規表現で取得

In [7]:
pathes = glob.glob("/Users/miyakooti/repositories/arai_MATLAB_program/csv/?_*/HRV_and_PLI.csv")
pathes.sort()
pathes

['/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/1_kim/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/2_souma/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/3_fujii/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/4_tubota/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/5_toki/HRV_and_PLI.csv']

### using all data

In [8]:
from scipy.stats import pearsonr
import statistics

In [18]:
for i,target_phase in enumerate(target_phases):
    
    for j,path in enumerate(pathes):

        df = pd.read_csv(path,index_col=0).fillna(0)

        df = df.loc[[target_phase]] # seriesとして取り出したいときはこっち
        if j == 0 and i == 0:
            print("flow_datasetを初期化しました")
            flow_dataset = df
        else:
            flow_dataset = pd.concat([flow_dataset, df], axis=0)

flow_dataset

flow_datasetを初期化しました


,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate,vlf,lf,hf,lf/hf,p_total,vlf_perc,lf_perc,hf_perc,lf_nu,hf_nu,lf_diff,hf_diff,lfhf_diff,kubios_lf,kubios_hf,kubios_lf/hf,FpZ,FC3,FC4,FCz,O1,O2,Fp2,mean_all,mean_3ch,concentration_FC3,concentration_FC4,concentration_FCz,concentration_mean,difficulty,fast,fulfillment,absorb,questionnaire_average
boredom,76.136223,788.061162,38.553302,16.680887,31.258471,0.582822,0.092025,22.0,22.103056,49.826426,3459.887136,0.443601,0.366667,518.685695,348.261920,297.243050,1.171640,1164.190666,44.553329,29.914509,25.532162,53.951857,46.048143,-9.955653,235.914887,-0.537107,318,315,1.000,0.28649,0.43996,0.43761,0.54694,0.17318,0.15961,0.23876,0.326079,0.474837,0.272161,0.092863,0.222104,0.195709,2,2.0,5.0,2.0,2.75
boredom,80.226665,747.881020,38.916159,14.717992,25.751214,0.423295,0.034091,24.0,18.208857,51.943439,2971.414676,0.350552,0.266667,231.299304,347.565192,358.816703,0.968643,937.681198,24.667158,37.066456,38.266386,49.203582,50.796418,115.133292,522.137564,-0.443419,334,359,0.929,0.34620,0.48977,0.61395,0.66071,0.37324,0.35980,0.31467,0.451191,0.588143,-0.027811,-0.110641,-0.077674,-0.072042,1,1.0,1.0,1.0,1.00
boredom,63.828817,940.014286,53.462669,24.390417,47.321822,0.752688,0.344086,32.0,33.461443,67.867019,7134.331888,0.493044,0.266667,624.630012,1216.676079,1012.413791,1.201758,2853.719882,21.888273,42.634741,35.476986,54.581742,45.418258,-285.030101,36.320655,-0.313405,934,1088,0.859,0.19084,0.36094,0.37935,0.43916,0.48950,0.44796,0.22319,0.361563,0.393150,0.051840,-0.010917,0.026382,0.022435,5,3.0,3.0,3.0,3.50
boredom,70.789522,847.583062,51.500818,18.344063,33.907024,0.607843,0.111111,28.0,23.975786,68.709213,5175.325957,0.348946,0.133333,457.939974,1233.866316,351.173620,3.513551,2042.979910,22.415295,60.395421,17.189284,77.844494,22.155506,-828.444322,24.417744,-2.434128,1296,309,4.190,0.34038,0.47183,0.38638,0.64152,0.28792,0.39155,0.28821,0.401113,0.499910,0.267063,0.372362,0.157989,0.265805,3,1.0,5.0,5.0,3.50
boredom,89.866583,667.656410,35.993944,17.595896,28.445049,0.455013,0.082262,26.0,20.113482,46.694019,2950.518808,0.430751,0.133333,95.834405,495.812750,206.007992,2.406765,797.655147,12.014516,62.158785,25.826699,70.646637,29.353363,135.824443,6.732313,0.562288,604,229,2.639,0.44885,0.60714,0.64297,0.73292,0.28617,0.39602,0.46605,0.511446,0.661010,-0.013874,0.047647,-0.007030,0.008914,3,2.0,4.0,2.0,2.75
boredom,71.332969,841.125786,39.674817,15.344297,24.405099,0.381703,0.037855,21.0,17.255790,53.203177,2884.179566,0.324338,0.133333,276.027700,830.045789,211.114474,3.931733,1317.187963,20.955832,63.016503,16.027665,79.723153,20.276847,401.674663,-72.458826,4.951572,963,202,4.770,0.34840,0.55059,0.56533,0.61080,0.30801,0.45166,0.36296,0.456821,0.575573,-0.013874,0.047647,-0.007030,0.008914,1,1.0,2.0,1.0,1.25
flow,79.897427,750.962861,43.874994,16.548756,29.365935,0.541322,0.084022,30.0,20.764207,58.349347,3806.284253,0.355860,0.128217,971.981731,207.806086,79.141757,2.625745,1258.929574,77.206998,16.506570,6.286432,72.419463,27.580537,130.500182,454.016180,-1.991212,290,80,3.610,0.26200,0.42757,0.43541,0.37519,0.66146,0.52691,0.25446,0.420429,0.412723,0.290671,0.095183,0.407868,0.264574,7,7.0,7.0,7.0,7.00
flow,82.444961,727.758242,32.812841,12.881806,24.531715,0.468553,0.017296,20.0,17.346535,43.077713,2347.552152,0.402680,0.129427,482.058147,127.708503,154.212651,0.828132,763.979302,63.098326,16.716226,20.185449,45.299369,54.700631,334.989981,726.741615,-0.302908,123,172,0.717,0.31442,0.42997,0.56499,0.58237,0.47581,0.43353,0.33135,0.447491,0.525777,0.031990,-0.061677,0.000665,-0.009674,7,5.0,5.0,6.0,5.75
flow,67.288214,891.686624,55.689848,20.200770,38.433417,0.677296,0.188776,32.0,27.176520,73.970311,6315.404293,0.367398,0.314299,1642.665400,539.936036,556.557245,0.970136,2739.158682,59.969706,19.711747,20.318547,49.242074,50.757926,391.709942,492.177201,-0.081783,375,461,0.815,0.38484,0.26153,0.31830,0.55653,0.23826,0.16282,0.33451,0.322399

# なんでこうなったのかわからんけど，難易度を数値に変換する

In [21]:
flow_dataset["difficulty"] = flow_dataset["difficulty"].astype("float")

In [22]:
flow_dataset.dtypes

bpm                      float64
ibi                      float64
sdnn                     float64
sdsd                     float64
rmssd                    float64
pnn20                    float64
pnn50                    float64
hr_mad                   float64
sd1                      float64
sd2                      float64
s                        float64
sd1/sd2                  float64
breathingrate            float64
vlf                      float64
lf                       float64
hf                       float64
lf/hf                    float64
p_total                  float64
vlf_perc                 float64
lf_perc                  float64
hf_perc                  float64
lf_nu                    float64
hf_nu                    float64
lf_diff                  float64
hf_diff                  float64
lfhf_diff                float64
kubios_lf                  int64
kubios_hf                  int64
kubios_lf/hf             float64
FpZ                      float64
FC3       

In [23]:
for survey_item in survey_items:
    

    # 保存用dict
    export_data = {    
        "target": target_columns,
        "mean": [],
        "SD": [],
        "R": [],
        "p": [],
    }

    # それぞれの説明変数に対して，，，
    for target_column in target_columns:
        print(f"{survey_item}に対する{target_column}の相関係数を計算します．")

        x = flow_dataset[target_column]
        y = flow_dataset[[survey_item]]

        # ピアソンの積率相関係数を計算        
        R, p = pearsonr(x, y)

        export_data["mean"].append(statistics.mean(x))
        export_data["SD"].append(statistics.pstdev(x))
        export_data["R"].append(R[0])
        export_data["p"].append(p)

    df = pd.DataFrame(export_data)

    # 保存
    save_path = f"/Users/miyakooti/repositories/arai_MATLAB_program/csv/{folder_name}/pearsonr_{survey_item}.csv"
    df.to_csv(save_path)  

difficultyに対するbpmの相関係数を計算します．
difficultyに対するibiの相関係数を計算します．
difficultyに対するlfの相関係数を計算します．
difficultyに対するhfの相関係数を計算します．
difficultyに対するlf/hfの相関係数を計算します．
difficultyに対するsdsdの相関係数を計算します．
difficultyに対するrmssdの相関係数を計算します．
difficultyに対するpnn20の相関係数を計算します．
difficultyに対するpnn50の相関係数を計算します．
difficultyに対するconcentration_FC3の相関係数を計算します．
difficultyに対するconcentration_FC4の相関係数を計算します．
difficultyに対するconcentration_FCzの相関係数を計算します．
difficultyに対するconcentration_meanの相関係数を計算します．
difficultyに対するFC3の相関係数を計算します．
difficultyに対するFC4の相関係数を計算します．
difficultyに対するFCzの相関係数を計算します．
difficultyに対するmean_3chの相関係数を計算します．
difficultyに対するlf_diffの相関係数を計算します．
difficultyに対するhf_diffの相関係数を計算します．
difficultyに対するlfhf_diffの相関係数を計算します．
difficultyに対するhr_madの相関係数を計算します．
difficultyに対するsd1の相関係数を計算します．
difficultyに対するsd2の相関係数を計算します．
difficultyに対するsの相関係数を計算します．
difficultyに対するsd1/sd2の相関係数を計算します．
difficultyに対するbreathingrateの相関係数を計算します．
difficultyに対するlf_nuの相関係数を計算します．
difficultyに対するhf_nuの相関係数を計算します．
fastに対するbpmの相関係数を計算します．
fastに対するibiの相関係数を計算します．
fastに対するlf

# 参考文献
- https://www.st-hakky-blog.com/entry/2018/01/30/004659